In [1]:
import os

%pwd

'c:\\Users\\satis\\Kidney-Disease-Classification-DP-Project\\research'

In [2]:
# exit from research folder to main project folder
os.chdir("../")
%pwd

'c:\\Users\\satis\\Kidney-Disease-Classification-DP-Project'

# Following workflows from README.md file

## 04. Update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


# below code from config.yaml 
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

## 05. Update the configuration manager in src config

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

## 06. Update components

In [9]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetchs data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## 07. Update the pipeline

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-03 19:58:06,358: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-03 19:58:06,363: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-03 19:58:06,365: INFO: common: Created directory at: artifacts]
[2024-07-03 19:58:06,365: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-07-03 19:58:06,366: INFO: 787829753: Downloading data from https://drive.google.com/file/d/1tMVSoSJua1xGXkMoqjEFBrICjt32zSKS/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1tMVSoSJua1xGXkMoqjEFBrICjt32zSKS
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1tMVSoSJua1xGXkMoqjEFBrICjt32zSKS&confirm=t&uuid=130b56ef-0a2a-44ef-9232-6c9e29ab51b4
To: c:\Users\satis\Kidney-Disease-Classification-DP-Project\artifacts\data_ingestion\data.zip
100%|██████████| 941M/941M [00:33<00:00, 28.2MB/s] 

[2024-07-03 19:58:41,588: INFO: 787829753: Downloaded data from https://drive.google.com/file/d/1tMVSoSJua1xGXkMoqjEFBrICjt32zSKS/view?usp=sharing into file artifacts/data_ingestion/data.zip]
